In [1]:
import numpy as np
import pandas as pd
import os
import time
import shutil

In [2]:
summary = pd.read_csv('summary.csv')

In [3]:
cohort = pd.read_csv('728 Cohort.csv')

In [4]:
summary.head()

,File Name,Folder,Participant ID,Study Date,Bar Code,Description,Image Size,KLG,Method,IsSuccessful,Month
0,9000296_96m_LEFT_KNEE.hdf5,/gpfs/data/denizlab/Datasets/OAI_original/96m/...,9000296,20120720,16604011302,LEFT,1024x1024,3.0,mix,1,96m
1,9000296_96m_RIGHT_KNEE.hdf5,/gpfs/data/denizlab/Datasets/OAI_original/96m/...,9000296,20120720,16604011302,RIGHT,1024x1024,2.0,mix,1,96m
2,9000798_96m_LEFT_KNEE.hdf5,/gpfs/data/denizlab/Datasets/OAI_original/96m/...,9000798,20120928,16603896401,LEFT,1024x1024,4.0,mix,1,96m
3,9000798_96m_RIGHT_KNEE.hdf5,/gpfs/data/denizlab/Datasets/OAI_original/96m/...,9000798,20120928,16603896401,RIGHT,1024x1024,1.0,mix,1,96m
4,9001897_96m_LEFT_KNEE.hdf5,/gpfs/data/denizlab/Datasets/OAI_original/96m/...,9001897,20130116,16603998101,LEFT,1024x1024,0.0,mix,1,96m


In [5]:
summary = summary.loc[summary.Month =='00m']

In [6]:
KLG = summary[['File Name','Participant ID','Study Date','Bar Code','Description','KLG']]

In [7]:
KLG.head()

,File Name,Participant ID,Study Date,Bar Code,Description,KLG
35944,9000296_00m_LEFT_KNEE.hdf5,9000296,20040729,175004,LEFT,3.0
35945,9000296_00m_RIGHT_KNEE.hdf5,9000296,20040729,175004,RIGHT,2.0
35946,9000798_00m_LEFT_KNEE.hdf5,9000798,20040802,162704,LEFT,4.0
35947,9000798_00m_RIGHT_KNEE.hdf5,9000798,20040802,162704,RIGHT,1.0
35948,9001400_00m_LEFT_KNEE.hdf5,9001400,20041202,410004,LEFT,1.0


In [8]:
KLG.KLG = KLG.KLG.astype(int)

/gpfs/share/skynet/apps/anaconda3/envs/tensorflow-env/lib/python3.6/site-packages/pandas/core/generic.py:4405: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [9]:
KLG['Description'].replace({'LEFT':0,'RIGHT':1},inplace = True)

/gpfs/share/skynet/apps/anaconda3/envs/tensorflow-env/lib/python3.6/site-packages/pandas/core/generic.py:5890: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


In [10]:
result = pd.merge(KLG,cohort,left_on=['Participant ID','Description'],right_on=['ID','R/L:1/0'])

In [11]:
result.head()

,File Name,Participant ID,Study Date,Bar Code,Description,KLG,ID,Label,R/L:1/0
0,9003380_00m_RIGHT_KNEE.hdf5,9003380,20041206,429603,1,1,9003380,0,1
1,9008884_00m_RIGHT_KNEE.hdf5,9008884,20050105,515203,1,2,9008884,0,1
2,9011053_00m_LEFT_KNEE.hdf5,9011053,20050214,562704,0,3,9011053,0,0
3,9013941_00m_RIGHT_KNEE.hdf5,9013941,20041209,435004,1,0,9013941,0,1
4,9014797_00m_RIGHT_KNEE.hdf5,9014797,20040812,186504,1,1,9014797,1,1


In [12]:
image_path = '/gpfs/data/denizlab/Users/bz1030/test/test_image'
target_path = '/gpfs/data/denizlab/Users/bz1030/test/kelvin_image'
images = os.listdir(image_path) 
rows = []
for idx,row in result.iterrows():
    part_id = row['Participant ID']
    study_date = row['Study Date']
    identifier = '{}_{}'.format(part_id,study_date)
    file_name = [i for i in images if identifier in i]
    if len(file_name) != 1:
        print(file_name)
        raise ValueError('Wrong ???')
    file_name = file_name[0]
    source_file = os.path.join(image_path,file_name)
    shutil.copy2(source_file,target_path)
    row['image_path'] = os.path.join(target_path,file_name)
    rows.append(row)
        
    


In [13]:
final_result = pd.DataFrame(rows)

In [14]:
final_result.head()

,File Name,Participant ID,Study Date,Bar Code,Description,KLG,ID,Label,R/L:1/0,image_path
0,9003380_00m_RIGHT_KNEE.hdf5,9003380,20041206,429603,1,1,9003380,0,1,/gpfs/data/denizlab/Users/bz1030/test/kelvin_i...
1,9008884_00m_RIGHT_KNEE.hdf5,9008884,20050105,515203,1,2,9008884,0,1,/gpfs/data/denizlab/Users/bz1030/test/kelvin_i...
2,9011053_00m_LEFT_KNEE.hdf5,9011053,20050214,562704,0,3,9011053,0,0,/gpfs/data/denizlab/Users/bz1030/test/kelvin_i...
3,9013941_00m_RIGHT_KNEE.hdf5,9013941,20041209,435004,1,0,9013941,0,1,/gpfs/data/denizlab/Users/bz1030/test/kelvin_i...
4,9014797_00m_RIGHT_KNEE.hdf5,9014797,20040812,186504,1,1,9014797,1,1,/gpfs/data/denizlab/Users/bz1030/test/kelvin_i...


In [15]:
final_result = final_result[['ID','Label','R/L:1/0','KLG','image_path','File Name']]

In [17]:
H5_PATH = '/gpfs/data/denizlab/Users/bz1030/data/OAI_processed/mix/00m/'
target_path ='/gpfs/data/denizlab/Users/bz1030/2Cem/H5Files'

In [19]:
for idx,row in final_result.iterrows():
    file_name = row['File Name']
    shutil.copy2(H5_PATH + file_name,target_path)
    

In [21]:
len(os.listdir(target_path))

728

In [41]:
final_result.to_csv('728_Cohort_KLG.csv',index = False)

In [42]:
final_result

,ID,Label,R/L:1/0,KLG,image_path,File Name
0,9003380,0,1,1,/gpfs/data/denizlab/Users/bz1030/test/kelvin_i...,9003380_00m_RIGHT_KNEE.hdf5
1,9008884,0,1,2,/gpfs/data/denizlab/Users/bz1030/test/kelvin_i...,9008884_00m_RIGHT_KNEE.hdf5
2,9011053,0,0,3,/gpfs/data/denizlab/Users/bz1030/test/kelvin_i...,9011053_00m_LEFT_KNEE.hdf5
3,9013941,0,1,0,/gpfs/data/denizlab/Users/bz1030/test/kelvin_i...,9013941_00m_RIGHT_KNEE.hdf5
4,9014797,1,1,1,/gpfs/data/denizlab/Users/bz1030/test/kelvin_i...,9014797_00m_RIGHT_KNEE.hdf5
5,9014883,1,0,3,/gpfs/data/denizlab/Users/bz1030/test/kelvin_i...,9014883_00m_LEFT_KNEE.hdf5
6,9015363,1,1,3,/gpfs/data/denizlab/Users/bz1030/test/kelvin_i...,9015363_00m_RIGHT_KNEE.hdf5
7,9015402,0,0,0,/gpfs/data/denizlab/Users/bz1030/test/kelvin_i...,9015402_00m_LEFT_KNEE.hdf5
8,9018291,0,0,1,/gpfs/data/denizlab/Users/bz1030/test/kelvin_i...,9018291_00m_LEFT_KNEE.hdf5
9,9027422,1,0,3,/gpfs/data/denizlab/Users/bz1030/test/kelvin_i...,9027422_00m_LEFT_KNEE.hdf5
